In [1]:
import codecs
import json
import re

同じ文字が連続する場合は長い方から処理する．
- ToDo: citationをどうにかする．

In [2]:
s = '# aa\n ## aaa\n `this` is ``` **sample string** for *extracting substring*. {cite:p}`Echeveste2020-sh` <a>'

In [3]:
def markdown2latex(s):
    s = re.sub(r'\###\ (.+?)\n', r'\\subsubsection{\1}', s)  # subsubsection
    s = re.sub(r'\##\ (.+?)\n', r'\\subsection{\1}', s)  # subsection
    s = re.sub(r'\#\ (.+?)\n', r'\\section{\1}', s)      # section
    
    s = re.sub(r'\*\*(.+?)\*\*', r'\\textbf{\1}', s)   # bold
    #s = re.sub(r'\*(.+?)\*', r'\\textit{\1}', s)       # italic
    
    s = s.replace(r"```{note}", "\\footnote{") # note to footnote
    s = s.replace(r"```", "}")
    s = re.sub(r'<(.+?)>', r'\\url{\1}', s) # url

    s = re.sub(r'{cite:p}`(.+?)`', r'\\cite{\1}', s)     
    s = re.sub(r'`(.+?)`', r'\\jl{\1}', s) # inline code with \newcommand{\jl}{\lstinline[language=julia]}

    s = re.sub(r'<(.+?)>', r'\\url{\1}', s) # url
    s = s.replace(r"\n", "\\")
    return s

In [4]:
markdown2latex(s)

'\\section{aa} \\subsection{aaa} \\jl{this} is } \\textbf{sample string} for *extracting substring*. \\cite{Echeveste2020-sh} \\url{a}'

変換

In [5]:
file_path = "../contents/bayesian-brain/neural-sampling.ipynb"

In [6]:
f = codecs.open(file_path, 'r', encoding="utf8")

In [7]:
source = f.read()
y = json.loads(source)

In [8]:
def all_remove(xlist, remove):
    return [value for value in xlist if value != remove]

In [19]:
num_cells = len(y['cells'])
master_list = []
for cell_idx in range(num_cells):
    cell = y['cells'][cell_idx]
    if cell['cell_type'] == 'markdown':
        # convert
        text = [markdown2latex(s) for s in cell['source']]
        text = all_remove(text, "\n")
        if ':filter: docname in docnames\n' in text:
            print(cell_idx)
        else:
            # save
            parted_file_path = "./text/test{}.tex".format(cell_idx)
            with open(parted_file_path, 'w', encoding='UTF-8') as f:
                f.writelines(text)
        master_list.append(r"\input{"+parted_file_path+"}\n")
    elif cell['cell_type'] == 'code':
        # ToDo:'outputs'
        code = cell['source']
        parted_file_path = "./text/test{}.jl".format(cell_idx)
        """
        if cell['outputs']:
            print(cell['outputs'][0])
            #'text/plain'
            #'image/png'
        """
        with open(parted_file_path, 'w', encoding='UTF-8') as f:
            f.writelines(code)
        master_list.append(r"\lstinputlisting[language=julia]{"+parted_file_path+"}\n")

49


In [20]:
file_path = './text/test.tex'
with open(file_path, 'w', encoding='UTF-8') as f:
        f.writelines(master_list)